In [71]:
from moviepy.config import change_settings
import os
from PIL import Image
from moviepy.editor import ImageSequenceClip, AudioFileClip, TextClip, CompositeVideoClip, VideoFileClip
from tkinter import filedialog, messagebox
import tkinter as tk
print("everything are loaded")

everything are loaded


In [72]:

# file:///C:/Program%20Files/ImageMagick-7.1.1-Q16-HDRI/
change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16\magick"})

In [73]:
# webp_image = Image.open('./media_input/The garden is full of blooming.webp')
# webp_image.convert('RGB').save('blooming.jpg', 'JPEG')

# print(webp_image)


In [74]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image
import os
from moviepy.editor import ImageSequenceClip, AudioFileClip, VideoFileClip, TextClip, CompositeVideoClip

class VideoEditorGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Video Editor for TikTok")
        self.root.geometry("500x200")
        self.root.configure(bg="white")  # Set background color to white
        self.setup_gui()
        self.selected_images = []
        self.selected_audio = None

    def setup_gui(self):
        tk.Button(self.root, text="Select Images", width=25, command=self.select_images).pack(pady=10)
        tk.Button(self.root, text="Select and Add Quote to it", width=25, command=self.add_quote).pack(pady=10)
        tk.Button(self.root, text="Select Sound", width=25, command=self.select_audio).pack(pady=10)
        tk.Button(self.root, text="Merge All", width=25, command=self.merge_media).pack(pady=10)
        tk.Button(self.root, text="Stop", width=25, command=self.root.destroy).pack(pady=10)

    def select_images(self):
        get_pics = filedialog.askopenfilenames(title="Select Images", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.webp")])
        
        output_directory = "input_medias"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        
        for get_pic in get_pics:
            if get_pic.lower().endswith('.webp'):
                im = Image.open(get_pic)
                new_pic = get_pic.rsplit('.', 1)[0] + '.png'
                im.save(new_pic, 'png')
                os.remove(get_pic)
                print(f"Converted and saved: {new_pic}")
                os.rename(new_pic, os.path.join(output_directory, os.path.basename(new_pic)))
                self.selected_images.append(os.path.join(output_directory, os.path.basename(new_pic)))
                messagebox.showinfo("Success", f"Converted and saved: {new_pic}")
            else:
                new_pic = get_pic
                os.rename(new_pic, os.path.join(output_directory, os.path.basename(new_pic)))
                self.selected_images.append(os.path.join(output_directory, os.path.basename(new_pic)))
                print(f"Saved: {new_pic}")
        
        messagebox.showinfo("Success", f"Images saved successfully!")

    def select_audio(self):
        get_audio = filedialog.askopenfilename(title="Select Audio", filetypes=[("Audio Files", "*.mp3;*.wav;*.ogg")])
        if not get_audio:
            return
        
        output_directory = "input_medias"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        
        new_audio_path = os.path.join(output_directory, os.path.basename(get_audio))
        os.rename(get_audio, new_audio_path)
        self.selected_audio = new_audio_path
        print(f"Selected Audio: {self.selected_audio}")
        messagebox.showinfo("Success", f"Audio added and saved successfully!")

    def add_quote(self):
        # Select an image to add the quote to
        image_file = filedialog.askopenfilename(title="Select an Image", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        if not image_file:
            return

        # Get the quote from the user
        quote = simpledialog.askstring("Input", "Enter your quote:")
        if not quote:
            return

        # Load the image
        image = Image.open(image_file)
        draw = ImageDraw.Draw(image) # type: ignore

        # Define font and size (make sure to have a font file accessible, e.g., arial.ttf)
        font_size = 20  # Adjust the size as needed
        font_path = "arial.ttf"  # You can specify the path to your font file
        try:
            font = ImageFont.truetype(font_path, font_size) # type: ignore
        except IOError:
            font = ImageFont.load_default() # type: ignore

        # Position for the quote (you can adjust this as needed)
        text_position = (10, 10)

        # Add text to the image
        draw.text(text_position, quote, font=font, fill="white")

        # Ensure the 'input_medias' directory exists
        output_directory = "input_medias"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        # Save the image with the quote to the 'input_medias' folder
        output_file = os.path.join(output_directory, os.path.basename(image_file))
        image.save(output_file)

        messagebox.showinfo("Success", f"Quote added to image and saved to {output_file} successfully!")

    def merge_media(self):
        if not self.selected_images:
            messagebox.showwarning("Warning", "No images selected to merge.")
            return
        
        output_directory = "input_medias"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        
        # Create a video from the selected images
        clip = ImageSequenceClip(self.selected_images, fps=1)  # 1 frame per second for simplicity
        
        if self.selected_audio:
            audio_clip = AudioFileClip(self.selected_audio)
            clip = clip.set_audio(audio_clip)
        
        output_file = os.path.join(output_directory, "merged_video.mp4")
        clip.write_videofile(output_file, codec="libx264")
        
        self.add_text_to_video(
            input_video_path=output_file,
            output_video_path=output_file,
            text="Follow me on TikTok: @pythonist",
            font_size=70,
            duration=10
        )
        
        messagebox.showinfo("Success", f"Video created and saved to {output_file} successfully!")

    def add_text_to_video(self, input_video_path, output_video_path, text, font_size=70, duration=10):
        # Load the video
        video = VideoFileClip(input_video_path)
        
        # Create a TextClip
        txt_clip = TextClip(text, fontsize=font_size, color='white', bg_color='black')
        
        # Set the duration of the text clip and position
        txt_clip = txt_clip.set_duration(duration).set_position('center').set_start((video.duration - duration) / 2)
        
        # Overlay the text clip on the video clip
        video_with_text = CompositeVideoClip([video, txt_clip])
        
        # Write the result to a file
        video_with_text.write_videofile(output_video_path, codec='libx264')

if __name__ == "__main__":
    root = tk.Tk()
    app = VideoEditorGUI(root)
    root.mainloop()


Saved: C:/Users/Admin/OneDrive/Desktop/GitHubx/mymoviecliper/input_medias/combined_image.jpg
Saved: C:/Users/Admin/OneDrive/Desktop/GitHubx/mymoviecliper/input_medias/showing determi.png
Selected Audio: input_medias\So-Sweet-Lish-Grooves.mp3
Moviepy - Building video input_medias\merged_video.mp4.
MoviePy - Writing audio in merged_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video input_medias\merged_video.mp4



Moviepy - Done !
Moviepy - video ready input_medias\merged_video.mp4
Moviepy - Building video input_medias\merged_video.mp4.
MoviePy - Writing audio in merged_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video input_medias\merged_video.mp4



Moviepy - Done !
Moviepy - video ready input_medias\merged_video.mp4


In [76]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageDraw, ImageFont
import os
from moviepy.editor import ImageSequenceClip, AudioFileClip, VideoFileClip, TextClip, CompositeVideoClip

class VideoEditorGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Video Editor for TikTok")
        self.root.geometry("500x300")
        self.root.configure(bg="white")  # Set background color to white
        self.setup_gui()
        self.selected_images = []
        self.selected_audio = None

    def setup_gui(self):
        tk.Button(self.root, text="Select Images", width=25, command=self.select_images).pack(pady=10)
        tk.Button(self.root, text="Select and Add Quote to it", width=25, command=self.add_quote).pack(pady=10)
        tk.Button(self.root, text="Select Sound", width=25, command=self.select_audio).pack(pady=10)
        tk.Button(self.root, text="Merge All", width=25, command=self.merge_media).pack(pady=10)
        tk.Button(self.root, text="Stop", width=25, command=self.root.destroy).pack(pady=10)
        
        # Add entry for text to add to the video
        tk.Label(self.root, text="Text to Add to Video:", bg="white").pack(pady=5)
        self.text_entry = tk.Entry(self.root, width=50)
        self.text_entry.pack(pady=5)

    def select_images(self):
        get_pics = filedialog.askopenfilenames(title="Select Images", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.webp")])
        
        output_directory = "input_medias"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        
        for get_pic in get_pics:
            if get_pic.lower().endswith('.webp'):
                im = Image.open(get_pic)
                new_pic = get_pic.rsplit('.', 1)[0] + '.png'
                im.save(new_pic, 'png')
                os.remove(get_pic)
                print(f"Converted and saved: {new_pic}")
                os.rename(new_pic, os.path.join(output_directory, os.path.basename(new_pic)))
                self.selected_images.append(os.path.join(output_directory, os.path.basename(new_pic)))
                messagebox.showinfo("Success", f"Converted and saved: {new_pic}")
            else:
                new_pic = get_pic
                os.rename(new_pic, os.path.join(output_directory, os.path.basename(new_pic)))
                self.selected_images.append(os.path.join(output_directory, os.path.basename(new_pic)))
                print(f"Saved: {new_pic}")
        
        messagebox.showinfo("Success", f"Images saved successfully!")

    def select_audio(self):
        get_audio = filedialog.askopenfilename(title="Select Audio", filetypes=[("Audio Files", "*.mp3;*.wav;*.ogg")])
        if not get_audio:
            return
        
        output_directory = "input_medias"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        
        new_audio_path = os.path.join(output_directory, os.path.basename(get_audio))
        os.rename(get_audio, new_audio_path)
        self.selected_audio = new_audio_path
        print(f"Selected Audio: {self.selected_audio}")
        messagebox.showinfo("Success", f"Audio added and saved successfully!")

    def add_quote(self):
        # Select an image to add the quote to
        image_file = filedialog.askopenfilename(title="Select an Image", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        if not image_file:
            return

        # Get the quote from the user
        quote = simpledialog.askstring("Input", "Enter your quote:")
        if not quote:
            return

        # Load the image
        image = Image.open(image_file)
        draw = ImageDraw.Draw(image)

        # Define font and size (make sure to have a font file accessible, e.g., arial.ttf)
        font_size = 20  # Adjust the size as needed
        font_path = "arial.ttf"  # You can specify the path to your font file
        try:
            font = ImageFont.truetype(font_path, font_size)
        except IOError:
            font = ImageFont.load_default()

        # Position for the quote (you can adjust this as needed)
        text_position = (10, 10)

        # Add text to the image
        draw.text(text_position, quote, font=font, fill="white")

        # Ensure the 'input_medias' directory exists
        output_directory = "input_medias"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        # Save the image with the quote to the 'input_medias' folder
        output_file = os.path.join(output_directory, os.path.basename(image_file))
        image.save(output_file)

        messagebox.showinfo("Success", f"Quote added to image and saved to {output_file} successfully!")

    def merge_media(self):
        if not self.selected_images:
            messagebox.showwarning("Warning", "No images selected to merge.")
            return
        
        output_directory = "input_medias"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        
        # Create a video from the selected images
        clip = ImageSequenceClip(self.selected_images, fps=1)  # 1 frame per second for simplicity
        
        if self.selected_audio:
            audio_clip = AudioFileClip(self.selected_audio)
            clip = clip.set_audio(audio_clip)
        
        output_file = os.path.join(output_directory, "merged_video.mp4")
        clip.write_videofile(output_file, codec="libx264")
        
        # Get text from the entry field
        text_to_add = self.text_entry.get()
        if text_to_add:
            self.add_text_to_video(
                input_video_path=output_file,
                output_video_path=output_file,
                text=text_to_add,
                font_size=70,
                duration=10,
            )
        
        messagebox.showinfo("Success", f"Video created and saved to {output_file} successfully!")

    def add_text_to_video(self, input_video_path, output_video_path, text, font_size=70, duration=10):
        # Load the video
        video = VideoFileClip(input_video_path)
        
        # Create a TextClip
        txt_clip = TextClip(text, fontsize=font_size, color='white', bg_color='black')
        
        # Set the duration of the text clip and position
        txt_clip = txt_clip.set_duration(duration).set_position('center').set_start((video.duration - duration) / 2)
        
        # Overlay the text clip on the video clip
        video_with_text = CompositeVideoClip([video, txt_clip])
        
        # Write the result to a file
        video_with_text.write_videofile(output_video_path, codec='libx264')
        
        print("add_text_to_video")

if __name__ == "__main__":
    root = tk.Tk()
    app = VideoEditorGUI(root)
    root.mainloop()

Saved: C:/Users/Admin/OneDrive/Desktop/GitHubx/mymoviecliper/input_medias/combined_image.jpg
Saved: C:/Users/Admin/OneDrive/Desktop/GitHubx/mymoviecliper/input_medias/showing determi.png
Selected Audio: input_medias\So-Sweet-Lish-Grooves.mp3
Moviepy - Building video input_medias\merged_video.mp4.
MoviePy - Writing audio in merged_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video input_medias\merged_video.mp4



Moviepy - Done !
Moviepy - video ready input_medias\merged_video.mp4
